In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict

from gensim import corpora, models
from gensim.models import Phrases
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

In [2]:
import spacy

In [3]:
import fr_core_news_lg

In [4]:
from gensim.corpora.dictionary import Dictionary

In [5]:
from gensim.models import LdaMulticore

In [32]:
df = pd.read_csv('french_dondata.csv')

In [35]:
from langdetect import detect_langs

In [37]:
languages = []

In [38]:
for row in range(len(df)):
    languages.append(detect_langs(df.iloc[row, 3]))

In [55]:
str(languages[1]).split(':')[0][1:]

'fr'

In [57]:
langs = [str(lang).split(':')[0][1:] for lang in languages]

In [56]:
for i in languages:
    str(languages[i]).split(':')[0][1:]

TypeError: list indices must be integers or slices, not list

In [58]:
df['langs'] = langs

In [59]:
df.head()

,_id,url,date,text,title,langs
0,5ec760320e5c92500d004aee,https://plus.lesoir.be/302282/article/2020-05-...,2020-05-22T04:22:52.000Z,Les concentrations de différents polluants rel...,La qualité de l’air en nette amélioration à Br...,fr
1,5ec7603216193c3860cf1616,https://www.rtbf.be/info/regions/detail_corona...,2020-05-22T04:37:03.000Z,L’amélioration de la qualité de l’air bruxello...,Coronavirus à Bruxelles : on respire mieux grâ...,fr
2,5ec76032039b233f9642aeb6,https://plus.lesoir.be/302121/article/2020-05-...,2020-05-20T19:36:26.000Z,La résolution pose plusieurs principes tel que...,La Chambre vote une résolution encadrant l’usa...,fr
3,5ec760320e5c92500d004af3,https://plus.lesoir.be/302273/article/2020-05-...,2020-05-21T21:01:32.000Z,Le nouveau coronavirus a officiellement touché...,Plus de cinq millions de cas de Covid-19 dans ...,fr
4,5ec7603216193c3860cf161f,https://www.rtbf.be/info/economie/detail_coron...,2020-05-22T04:00:00.000Z,Les résultats d’une enquête menée par le prest...,Coronavirus en Belgique : les ouvriers plus to...,fr


In [67]:
df['langs'].unique()

array(['fr', 'nl', 'af', 'es', 'en', 'de', 'bg'], dtype=object)

In [68]:
df.loc[df['langs'] == 'nl'].count()

_id      8341
url      8341
date     8341
text     8341
title    8341
langs    8341
dtype: int64

In [72]:
df.loc[df['langs'] == 'es'].head()

,_id,url,date,text,title,langs
2256,5ee851fde19d4149947f48e8,https://www.hln.be/de-krant/waar-halen-ze-de-c...,2020-06-15T22:00:00.000Z,"sport, sportdiscipline, Andrew, Barcelona, CCC...",Waar halen ze de centen?,es
2267,5ee88a3c25eb8a005251b29c,https://www.hln.be/sport/wielrennen/nieuwe-spo...,2020-06-16T05:47:00.000Z,"sport, sportdiscipline, Andrew, Barcelona, CCC...",Nieuwe sponsor van Mitchelton-Scott roept vrag...,es


In [63]:
df_fr = df.loc[df['langs'] == 'fr']

In [64]:
df_fr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69024 entries, 0 to 77380
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   _id     69024 non-null  object
 1   url     69024 non-null  object
 2   date    69024 non-null  object
 3   text    69024 non-null  object
 4   title   69024 non-null  object
 5   langs   69024 non-null  object
dtypes: object(6)
memory usage: 3.7+ MB


In [34]:
df.head()


,_id,url,date,text,title
0,5ec760320e5c92500d004aee,https://plus.lesoir.be/302282/article/2020-05-...,2020-05-22T04:22:52.000Z,Les concentrations de différents polluants rel...,La qualité de l’air en nette amélioration à Br...
1,5ec7603216193c3860cf1616,https://www.rtbf.be/info/regions/detail_corona...,2020-05-22T04:37:03.000Z,L’amélioration de la qualité de l’air bruxello...,Coronavirus à Bruxelles : on respire mieux grâ...
2,5ec76032039b233f9642aeb6,https://plus.lesoir.be/302121/article/2020-05-...,2020-05-20T19:36:26.000Z,La résolution pose plusieurs principes tel que...,La Chambre vote une résolution encadrant l’usa...
3,5ec760320e5c92500d004af3,https://plus.lesoir.be/302273/article/2020-05-...,2020-05-21T21:01:32.000Z,Le nouveau coronavirus a officiellement touché...,Plus de cinq millions de cas de Covid-19 dans ...
4,5ec7603216193c3860cf161f,https://www.rtbf.be/info/economie/detail_coron...,2020-05-22T04:00:00.000Z,Les résultats d’une enquête menée par le prest...,Coronavirus en Belgique : les ouvriers plus to...


1. Filtering and tokenizing


In [74]:
key_words = ['données', 'data', 'sport','politique' ]

In [77]:
test_a = []
for row in range(len(df_fr)):
    if any(word in str(df_fr.iloc[row, 3]) for word in key_words):
        test_a.append(str(df_fr.iloc[row, 3]))

In [78]:
len(test_a)

68511

In [52]:
for i in list: 
    str(i)

In [79]:
nlp = fr_core_news_lg.load() #spacy

In [80]:
remove_tagged= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']


In [85]:
test_a[1]

'L’amélioration de la qualité de l’air bruxellois au terme d’un mois et demi de confinement se traduit par une baisse des concentrations de monoxyde d’azote allant jusqu’à 75% dans les sites habituellement fortement exposés aux émissions du trafic automobile, selon des rapports de Bruxelles Environnement présentés vendredi par le cabinet du ministre régional Alain Maron.\\n\\nLes concentrations de polluants relevées dans différentes stations de mesures caractéristiques d’un certain trafic (comme la station Arts-Loi ou la station Uccle) du 19 mars au 19 avril, puis sur une période étendue au 3 mai dans un second rapport, ont été comparées aux données de 2010-2019. Les conditions météorologiques, qui influent sur la dispersion des polluants, ont été considérées comme similaires, voire légèrement meilleures, à celle d’un mois de mars ou d’avril "normal".\\n\\n►►► À lire aussi : Meurtres, accidents de la route, pollution… En Asie, le confinement a sauvé des vies\\n\\nDans les stations fort

In [83]:
list = test_a[:9999]

In [84]:
tokens = []
for text in nlp.pipe(list):
   proj_tok = [token.lemma_.lower() for token in text if token.pos_ not in remove_tagged and not token.is_stop and token.is_alpha]
   tokens.append(proj_tok)

In [86]:
tokens[1][:3]

['amélioration', 'qualité', 'air']

In [87]:
test_tokens = tokens

In [91]:
test_tokens[1][:3]

['amélioration', 'qualité', 'air']

#Removing infrequent words
frequency = defaultdict(int)
for text in test_tokens:
    for token in text:
        frequency[token] += 1


test_tokens = [[token for token in text if frequency[token] > 1]
         for text in test_tokens]

In [83]:
#Removing infrequent words
frequency = defaultdict(int)
for text in test_tokens:
    for token in text:
        frequency[token] += 1


test_tokens = [[token for token in text if frequency[token] > 1]
         for text in test_tokens]

In [86]:
len(tokens[1])
len(test_tokens[1])

596

In [110]:
#Adding bigrams
bigram = Phrases(test_tokens, min_count=20) #gensim
for idx in range(len(test_tokens)):
    for token in bigram[test_tokens[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            test_tokens[idx].append(token)

In [108]:
len(tokens[55])

115

In [111]:
len(test_tokens[55])

122

In [124]:
test_dictionary = Dictionary(test_tokens)

In [125]:
test_dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)

In [126]:
test_corpus = [test_dictionary.doc2bow(doc) for doc in test_tokens]

In [115]:
#Tf-Idf for word frequency filter 
\\\\\\\\\\\
from gensim.models.tfidfmodel import TfidfModel
tfidf = TfidfModel(test_corpus)
tfidf = TfidfModel(test_corpus, id2word = test_dictionary)
low_value = 0.2
low_value_words = []
for bow in test_corpus:
    low_value_words += [id for id, value in tfidf[bow] if value < low_value]
test_dictionary.filter_tokens(bad_ids=low_value_words) \\\\\\\\\\\\\\\\\
new_corpus = [test_dictionary.doc2bow(doc) for doc in test_tokens]

In [127]:
len(test_corpus[1])

81

In [131]:
test_lda = LdaMulticore(corpus=test_corpus, id2word=test_dictionary, iterations=50, num_topics=4, workers = 4, passes=10)

In [132]:
import pickle
import pyLDAvis.gensim

LDAvis = pyLDAvis.gensim.prepare(test_lda, test_corpus, test_dictionary)
path = os.path.join('.results')

with open(path, 'wb') as f:
    pickle.dump(LDAvis,f)

with open(path, 'rb') as f:
    LDAvis = pickle.load(f)

pyLDAvis.save_html(LDAvis, './results' + '_french_data_thursday' + '.html')